# Pairwise FST

In [6]:
using Pkg ; Pkg.instantiate()
using PopGen
using DataFrames
using CSV
using MultipleTesting

## Load the data

In [7]:
bft = PopGen.read("../data/bft.kinrm.gen", silent = true)

PopData{Diploid, 1448 SNP loci}
  Samples: 326
  Populations: 9

Rename populations, since genepop files don't preserve population name

In [11]:
popdict = Dict(
    "1" => "BRZ",
    "2" => "BRZSP",
    "3" => "KEY",
    "4" => "MRT",
    "5" => "PNS",
    "6" => "PR",
    "7" => "SCA",
    "8" => "TX",
    "9" => "VZ",
)
populations!(bft, popdict)
populations(bft, counts = true)

,population,count
,String,Int64
1,BRZ,23
2,BRZSP,14
3,KEY,55
4,MRT,40
5,PNS,30
6,PR,38
7,SCA,52
8,TX,28
9,VZ,46


## Summary Information

In [12]:
summarystats(bft)

,Het_obs,HS,HT,DST,HT′,DST′,FST,FST′,FIS
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0778,0.0823,0.0824,0.0002,0.0825,0.0002,0.0021,0.0024,0.0544


## Hudson pairwise FST

Calculate pairwise FST without significance testing

In [15]:
pairwisefst(bft, method = "Hudson")

Pairwise FST: Hudson et al. 1992
        BRZ         BRZSP       KEY         MRT         PNS          PR        ⋯
────────────────────────────────────────────────────────────────────────────────
   BRZ  0.0         0.0         0.0         0.0         0.0          0.0       ⋯
 BRZSP  0.00306326  0.0         0.0         0.0         0.0          0.0
   KEY  0.00232454  0.00451952  0.0         0.0         0.0          0.0
   MRT  0.0028456   0.00442876  0.00119415  0.0         0.0          0.0
   PNS  0.00178032  0.00204233  0.00119187  0.00110536  0.0          0.0       ⋯
    PR  0.00106353  0.00250232  0.00182457  0.00224379  0.00121787   0.0
   SCA  0.00152872  0.00397563  0.00111994  0.00157691  0.00136468   0.0011928
    TX  0.0052479   0.00612004  0.00445504  0.00226417  0.00501863   0.0049077
    VZ  0.00184771  0.00323136  0.00145034  0.00144044  0.000960011  0.0006706 ⋯
                                                               4 columns omitted

Calculate pairwise FST with significance testing

In [ ]:
pfst = pairwisefst(bft, method = "Hudson", iterations = 999)

In [30]:
CSV.write("hudson.fst", pfst.results)
pfst.results

,BRZ,BRZSP,KEY,MRT,PNS,PR,SCA
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.029029,0.037037,0.01001,0.0920921,0.624625,0.455455
2,0.00306326,0.0,0.00700701,0.00700701,0.288288,0.404404,0.0680681
3,0.00232454,0.00451952,0.0,0.0600601,0.225225,0.00600601,0.031031
4,0.0028456,0.00442876,0.00119415,0.0,0.182182,0.00600601,0.012012
5,0.00178032,0.00204233,0.00119187,0.00110536,0.0,0.236236,0.191191
6,0.00106353,0.00250232,0.00182457,0.00224379,0.00121787,0.0,0.133133
7,0.00152872,0.00397563,0.00111994,0.00157691,0.00136468,0.00119284,0.0
8,0.0052479,0.00612004,0.00445504,0.00226417,0.00501863,0.00490776,0.00413617
9,0.00184771,0.00323136,0.00145034,0.00144044,0.000960011,0.000670604,0.000866163


Function pulled from `PopGenCore.jl` to isolate the top triangle of the output matrix for P-value correction

In [20]:
function partitionarray(array::AbstractArray, steps::AbstractVector{<:Integer})
    v = axes(array,1)
    v == 1:sum(steps) || error("Steps provided do not sum to length of the first dimension")
    i = firstindex(v)
    tmp = (view(v, i:(i+=s)-1) for s in steps)
    [view(array,r,:) for r in tmp]
end

partitionarray (generic function with 1 method)

Function to perform a Benjamini-Hochberg FDR correction on the P-values in the upper triangle

In [21]:

function adjustpval(fstval::DataFrame)
    fst = deepcopy(fstval)
    rows = size(fst,1)
    pval = mapreduce(vcat, 1:rows-1) do i
        collect(fst[i,i+1:end])
    end
    posthoc = round.(adjust(pval, BenjaminiHochberg()),digits = 4)
    splitpart = partitionarray(posthoc, reverse(collect(1:(rows-1)))) .|> collect
    for i in 1:(rows-1)
        fst[i, (i+1):end] .= splitpart[i][:,1]
    end
    insertcols!(fst, 1, :pop => names(fst))
    return fst
end

adjustpval (generic function with 1 method)

## Correct the P-values for multiple testing

In [25]:
adjusted_pfst = adjustpval(pfst.results)

,pop,BRZ,BRZSP,KEY,MRT,PNS,PR
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,BRZ,0.0,0.0653,0.0741,0.03,0.1507,0.6246
2,BRZSP,0.00306326,0.0,0.0229,0.0229,0.3348,0.455
3,KEY,0.00232454,0.00451952,0.0,0.1135,0.2896,0.0229
4,MRT,0.0028456,0.00442876,0.00119415,0.0,0.2623,0.0229
5,PNS,0.00178032,0.00204233,0.00119187,0.00110536,0.0,0.2933
6,PR,0.00106353,0.00250232,0.00182457,0.00224379,0.00121787,0.0
7,SCA,0.00152872,0.00397563,0.00111994,0.00157691,0.00136468,0.00119284
8,TX,0.0052479,0.00612004,0.00445504,0.00226417,0.00501863,0.00490776
9,VZ,0.00184771,0.00323136,0.00145034,0.00144044,0.000960011,0.000670604


In [27]:
CSV.write("hudson.fdr.fst", adjusted_pfst)

"hudson.fdr.fst"